#### Studenti:
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Mihai Lacusteanu
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Alexandru-Florian Ion 
#### Profesor: 
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Victor-Emil Neagoe


Baza de date: https://www.cs.toronto.edu/~kriz/cifar.html





Imagini si clasele lor:

![title](all3.png)


In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
 
    
batch_size = 300
nr_epoci = 50


def rata_de_invatare_controlata(epoch):
    lrate = 0.005
    if epoch > 10:
        lrate = 0.001
    if epoch > 20:
        lrate = 0.0001
    if epoch > 30:
        lrate = 0.00005
    if epoch > 40:
        lrate = 0.00001
    return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2

In [2]:
#augmentare imagini
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 
#antrenare

 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=nr_epoci,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(rata_de_invatare_controlata)])

 


Instructions for updating:
Use tf.cast instead.
Epoch 1/50
166/166 [==============================] - 21s 126ms/step - loss: 2.9032 - acc: 0.2955 - val_loss: 1.7766 - val_acc: 0.4087
Epoch 2/50
166/166 [==============================] - 17s 105ms/step - loss: 1.7534 - acc: 0.4409 - val_loss: 1.6600 - val_acc: 0.4354
Epoch 3/50
166/166 [==============================] - 17s 105ms/step - loss: 1.6028 - acc: 0.4975 - val_loss: 1.4540 - val_acc: 0.5047
Epoch 4/50
166/166 [==============================] - 18s 106ms/step - loss: 1.5502 - acc: 0.5245 - val_loss: 2.2082 - val_acc: 0.3998
Epoch 5/50
166/166 [==============================] - 18s 106ms/step - loss: 1.4635 - acc: 0.5515 - val_loss: 1.4151 - val_acc: 0.5616
Epoch 6/50
166/166 [==============================] - 18s 106ms/step - loss: 1.3688 - acc: 0.5796 - val_loss: 1.1407 - val_acc: 0.6516
Epoch 7/50
166/166 [==============================] - 18s 109ms/step - loss: 1.3125 - acc: 0.6062 - val_loss: 1.5217 - val_acc: 0.5269
Epoch 8

In [3]:
#salvare model
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%d%m%Y_%H%M%S")
model_name = 'model_' + date_time

model_json = model.to_json()
with open(model_name + '.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights(model_name + '.h5') 

In [4]:
#testare
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nAcuratete: %.3f Functie Cost: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 67us/step

Acuratete: 81.800 Functie Cost: 0.653


In [5]:
from sklearn.metrics import confusion_matrix

In [6]:
predictions = model.predict(x_test)
y_pred = (predictions > 0.5)
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [7]:
matrix

array([[872,  18,  14,   5,   7,   1,   7,   5,  36,  35],
       [ 15, 922,   0,   0,   0,   0,   2,   0,   6,  55],
       [185,   3, 612,   9,  54,  13,  97,  12,   3,  12],
       [218,   8,  18, 498,  41,  64, 100,  18,  14,  21],
       [111,   1,  15,   9, 757,   1,  73,  30,   2,   1],
       [201,   2,   8,  69,  29, 599,  38,  45,   0,   9],
       [ 47,   0,   5,   9,   4,   0, 932,   2,   1,   0],
       [ 76,   1,   3,   4,  23,   5,  13, 864,   1,  10],
       [ 67,  13,   0,   1,   0,   0,   5,   1, 883,  30],
       [ 29,  31,   1,   1,   0,   0,   1,   1,   5, 931]], dtype=int64)

### Observatii:
Setul de date contine imagini in care obiectele sunt de marimi si inclinatii diferite asa ca augumentarea imaginilor imbunatateste acuratetea cu un procent de 10%
Imaginile sunt destul de complexe ceea ce implica o structura cu multe straturi si multi neuroni pe fiecare strat, ceea ce la randul ei implica un set de date mai mare decat cel pe care il avem la dispozitie 
